# Read write various geo data format

To be able to use sedona to do geospatial operations (e.g calculate distance, area hierarchy, etc.), we need to construct geo dataframe first. A geo dataframe contains one or more columns of below type:
- Point : a point on the map with a (x,y) coordinates
- Line: two point which can form a line
- Polygon: a list of point which can form a polygon

The **full list of the constructor for the geo data types** can be found [here](https://sedona.apache.org/1.4.1/api/sql/Constructor/)

In [2]:
from sedona.spark import *

In [3]:
# build a sedona session (sedona >= 1.4.1)
config = SedonaContext.builder(). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.4.1,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2'). \
    getOrCreate()

# create a sedona context
sedona = SedonaContext.create(config)

23/07/20 14:55:34 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
23/07/20 14:55:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/pengfei/opt/spark-3.3.0/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pengfei/.ivy2/cache
The jars for the packages stored in: /home/pengfei/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-724bfcd3-2274-4099-81bf-67e509fd8cac;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 612ms :: artifacts dl 31ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------

23/07/20 14:55:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Read from CSV/TSV of plain text string

In below example, we will read a normal csv file which contains two column x, y. You can notice the content of the csv is `plain text` string.

### 1.1 Point example

In below example, we will construct a geo dataframe which contains a **Point** column

In [4]:
file_path="../../../../data/test_points.csv"

In [5]:
# read a normal csv
test_df = sedona.read.format("csv").\
          option("delimiter",",").\
          option("header","false").\
          load(file_path)

test_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [6]:
test_df.show(5)

+---+-----+
|_c0|  _c1|
+---+-----+
|1.1|101.1|
|2.1|102.1|
|3.1|103.1|
|4.1|104.1|
|5.1|105.1|
+---+-----+
only showing top 5 rows



In [7]:
# create a temp view
test_df.createOrReplaceTempView("raw_table")

In [8]:
point_df = sedona.sql("select ST_Point(cast(raw_table._c0 as Decimal(24,20)), cast(raw_table._c1 as Decimal(24,20))) as point from raw_table")

In [9]:
point_df.show(5)

+-----------------+
|            point|
+-----------------+
|POINT (1.1 101.1)|
|POINT (2.1 102.1)|
|POINT (3.1 103.1)|
|POINT (4.1 104.1)|
|POINT (5.1 105.1)|
+-----------------+
only showing top 5 rows



In [10]:
point_df.printSchema()

root
 |-- point: geometry (nullable = true)



> You can notice that, we used the constructor ST_Point() to build the point column.



In [ ]:
### 1.3 Polygon example

## 2. Read from TSV

